In [1]:
from datetime import datetime
import json
from tab_dataset import Cdataset
import pandas as pd
import ntv_pandas as npd
import pathlib


## initialisation des données
- lecture du fichier issu de l'api

In [5]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04', '2023-10-21', '2023-10-31', '2023-11-23', '2023-12-14', '2023-12-29',
         '2024-01-15', '2024-02-07', '2024-02-28', '2024-03-27', '2024-04-24', '2024-05-19', '2024-06-05', '2024-08-02',
         '2024-10-24', '2024-12-04']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.3.1-'+date[:4]+date[5:7]+date[8:]+'.csv', # après 2024-02-06
    'chemin': str(pathlib.Path(npd.__file__).parent.parent.parent/"Environmental-Sensing"/"python"/"Validation"/"irve"/"Analyse")
      }
try:
    irve = pd.read_csv(log['chemin'] + '/' + log['file'], sep=',', low_memory=False, dtype='object')
except:
    log['chemin'] = 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Validation\\irve\\Analyse'
    irve = pd.read_csv(log['chemin'] + '/' + log['file'], sep=',', low_memory=False, dtype='object')
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 
irve.columns

nombre de lignes :  116625


Index(['nom_amenageur', 'siren_amenageur', 'contact_amenageur',
       'nom_operateur', 'contact_operateur', 'telephone_operateur',
       'nom_enseigne', 'id_station_itinerance', 'id_station_local',
       'nom_station', 'implantation_station', 'adresse_station',
       'code_insee_commune', 'coordonneesXY', 'nbre_pdc', 'id_pdc_itinerance',
       'id_pdc_local', 'puissance_nominale', 'prise_type_ef', 'prise_type_2',
       'prise_type_combo_ccs', 'prise_type_chademo', 'prise_type_autre',
       'gratuit', 'paiement_acte', 'paiement_cb', 'paiement_autre',
       'tarification', 'condition_acces', 'reservation', 'horaires',
       'accessibilite_pmr', 'restriction_gabarit', 'station_deux_roues',
       'raccordement', 'num_pdl', 'date_mise_en_service', 'observations',
       'date_maj', 'cable_t2_attache', 'last_modified', 'datagouv_dataset_id',
       'datagouv_resource_id', 'datagouv_organization_or_owner', 'created_at',
       'consolidated_longitude', 'consolidated_latitude',
     

In [3]:
# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

In [4]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance','restriction_gabarit']] = data[['id_station_itinerance','id_pdc_itinerance','restriction_gabarit']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'], 'soit : ', len(non_concerne)/(len(non_concerne)+len(itinerance)))
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  1134 soit :  0.009723472668810289
nombre de pdc en itinerance   :  115491


In [13]:
nom_op = list(itinerance['nom_operateur'].unique())
tesla = 'TESLA France SARL'

In [15]:
stat_tesla = itinerance[itinerance.nom_operateur == tesla]
len(stat_tesla)

3057

In [17]:
stat_tesla.to_csv('stations_tesla.csv')

In [16]:
stat_tesla

,index,nom_amenageur,siren_amenageur,contact_amenageur,nom_operateur,contact_operateur,telephone_operateur,nom_enseigne,id_station_itinerance,id_station_local,...,datagouv_resource_id,datagouv_organization_or_owner,created_at,consolidated_longitude,consolidated_latitude,consolidated_code_postal,consolidated_commune,consolidated_is_lon_lat_correct,consolidated_is_code_insee_verified,non_concerne
102248,103134,TESLA France SARL,524335262,charging-france@tesla.com,TESLA France SARL,charging-france@tesla.com,0970732103,TESLA France SARL,FRTSLP1755,1755,...,9a53f96f-9bb7-4d94-9687-4c8525dea018,tesla-motors,2023-12-14T11:38:33.809000+00:00,7.388186,47.782002,68390,Sausheim,True,True,False
102249,103135,TESLA France SARL,524335262,charging-france@tesla.com,TESLA France SARL,charging-france@tesla.com,0970732103,TESLA France SARL,FRTSLP1755,1755,...,9a53f96f-9bb7-4d94-9687-4c8525dea018,tesla-motors,2023-12-14T11:38:33.809000+00:00,7.388186,47.782002,68390,Sausheim,True,True,False
102250,103136,TESLA France SARL,524335262,charging-france@tesla.com,TESLA France SARL,charging-france@tesla.com,0970732103,TESLA France SARL,FRTSLP1755,1755,...,9a53f96f-9bb7-4d94-9687-4c8525dea018,tesla-motors,2023-12-14T11:38:33.809000+00:00,7.388186,47.782002,68390,Sausheim,True,True,False
102251,103137,TESLA France SARL,524335262,charging-france@tesla.com,TESLA France SARL,charging-france@tesla.com,0970732103,TESLA France SARL,FRTSLP1755,1755,...,9a53f96f-9bb7-4d94-9687-4c8525dea018,tesla-motors,2023-12-14T11:38:33.809000+00:00,7.388186,47.782002,68390,Sausheim,True,True,False
102252,103138,TESLA France SARL,524335262,charging-france@tesla.com,TESLA France SARL,charging-france@tesla.com,0970732103,TESLA France SARL,FRTSLP1755,1755,...,9a53f96f-9bb7-4d94-9687-4c8525dea018,tesla-motors,2023-12-14T11:38:33.809000+00:00,7.388186,47.782002,68390,Sausheim,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106518,107535,TESLA France SARL,524335262,charging-france@tesla.com,TESLA France SARL,charging-france@tesla.com,0970732103,TESLA France SARL,FRTSLP29935,29935,...,9a53f96f-9bb7-4d94-9687-4c8525dea018,tesla-motors,2023-12-14T11:38:33.809000+00:00,6.365683,45.655436,73200,Albertville,True,True,False
106519,107536,TESLA France SARL,524335262,charging-france@tesla.com,TESLA France SARL,charging-france@tesla.com,0970732103,TESLA France SARL,FRTSLP29935,29935,...,9a53f96f-9bb7-4d94-9687-4c8525dea018,tesla-motors,2023-12-14T11:38:33.809000+00:00,6.365683,45.655436,73200,Albertville,True,True,False
106520,107537,TESLA France SARL,524335262,charging-france@tesla.com,TESLA France SARL,charging-france@tesla.com,0970732103,TESLA France SARL,FRTSLP29935,29935,...,9a53f96f-9bb7-4d94-9687-4c8525dea018,tesla-motors,2023-12-14T11:38:33.809000+00:00,6.365683,45.655436,73200,Albertville,True,True,False
106521,107538,TESLA France SARL,524335262,charging-france@tesla.com,TESLA France SARL,charging-france@tesla.com,0970732103,TESLA France SARL,FRTSLP29935,29935,...,9a53f96f-9bb7-4d94-9687-4c8525dea018,tesla-motors,2023-12-14T11:38:33.809000+00:00,6.365683,45.655436,73200,Albertville,True,True,False
